In [1]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

# Para clustering
from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

**Ejercicio 1**

Trabajamos con las variables "bill_length_mm" y "bill_depth_mm" de pinguinos.

1. Escalar las variables por MinMax (KNN es sensible a las escalas)
2. Para cada uno de los pinguinos 15, 151 y 313 (ver slides), numerar en un grafico los 9 pinguinos mas cercanos, ordenados de más cerca a más lejos.
3. Clasificar cada pingüino utilizando KNN con K = 1, 3 y 9.

In [2]:
# Utilizamos NearestNeighbors para obtener los vecinos más cercanos
from sklearn.neighbors import NearestNeighbors

In [3]:
# Eliminamos datos faltantes y reseteamos los índices, para no tener problemas al graficar
penguins = sns.load_dataset("penguins").dropna().reset_index(drop=True)

# Normalizamos las variables "bill_length_mm" y "bill_depth_mm" por MinMax
penguins[["bill_length_mm", "bill_depth_mm"]] = MinMaxScaler().fit_transform(penguins[["bill_length_mm", "bill_depth_mm"]])

penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,0.254545,0.666667,181.0,3750.0,Male
1,Adelie,Torgersen,0.269091,0.511905,186.0,3800.0,Female
2,Adelie,Torgersen,0.298182,0.583333,195.0,3250.0,Female
3,Adelie,Torgersen,0.167273,0.738095,193.0,3450.0,Female
4,Adelie,Torgersen,0.261818,0.892857,190.0,3650.0,Male


In [4]:
# Nos quedamos solo con largo y profundidad del pico
datos = penguins[["bill_depth_mm", "bill_length_mm"]]

In [7]:
# Veamos primero los 9 vecinos más cercanos del pingüino 15
K = 9
ind = 15
neighbors = NearestNeighbors(n_neighbors=K+1)  # Esta función nos devuelve los más cercanos incluyendo a si mismo, por eso tomamos 10.
neighbors.fit(datos)  # En el ajuste solamente almacenamos los datos

# Ahora podemos buscar los vecinos más cercanos a un punto cualquiera o un conjunto de puntos.
# Tenemos que pasarle un DataFrame
distances, indices = neighbors.kneighbors(penguins.iloc[[ind]])

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- body_mass_g
- flipper_length_mm
- island
- sex
- species


In [ ]:
# Nos devuelve un vector de distancias (opcional)
distances.flatten()

Vemos que las distancias están ordenadas de menor a mayor.

In [ ]:
# Y un vector de índices

In [ ]:
indices.flatten()

In [ ]:
# Graficamos
(
    so.Plot(data = penguins.iloc[indices.flatten()], x = "bill_length_mm", y = "bill_depth_mm",  text = np.arange(K+1).astype(str))
    .add(so.Text(valign = "bottom"))
    .add(so.Dot(), color = "species")
    .add(so.Dot(color = "red"), data = penguins.iloc[[ind]], x = "bill_length_mm", y = "bill_depth_mm")
)

**Ejercicio 2**

1. Implementar una función que reciba un DataFrame (que tenga solo las variables numéricas a utilizar para medir distancias), un vector de categorías (indexado igual que el DataFrame), un índice y un valor de K y devuelva la predicción por K-means para el dato indicado. Importante: debemos ignorar al propio dato en la votación.
2. Aplicar la función a los datos de pingüinos.

**Sugerencia:** para elegir la categoría más votada podemos calcular la moda. El paquete statistics provee el comando `mode`.


In [ ]:
# Cargamos el comando mode
import statistics
from statistics import mode

In [ ]:
# Item 1

# Antes de hacer la función lo calculamos para un ejemplo.
K = 9
ind = 151
datos = penguins[["bill_depth_mm", "bill_length_mm"]]
categorias = penguins["species"]

neighbors = NearestNeighbors(n_neighbors= K)  # Vamos a eliminar al propio pinguino de los datos, por eso tomamos K.
neighbors.fit(???)

distances, indices = neighbors.kneighbors(???)
votos = categorias[indices.flatten()]

In [ ]:
votos

In [ ]:
# El mas votado
???

In [ ]:
# Juntamos todo en una función
def mas_votado(datos, categorias, ind, K):
    neighbors = NearestNeighbors(n_neighbors= K)  # Vamos a eliminar al propio pinguino de los datos, por eso tomamos K.
    neighbors.fit(datos.drop([ind]))

    distances, indices = neighbors.kneighbors(datos.iloc[[ind]])
    votos = categorias[indices.flatten()]

    return(mode(votos))

In [ ]:
# Item 2 - Aplicamos la función a los datos de pingüinos
datos = penguins[["bill_depth_mm", "bill_length_mm"]]
categorias = penguins["species"]
mas_votado(datos, categorias, 313, 9)

**Ejercicio 3**
1. Implementar una función que reciba un DataFrame, un vector de categorías y un valor de K y calcule las predicciones para todos los datos y nos devuelva el porcentaje de aciertos.
2. Aplicar la función a los datos de pingüinos.

In [ ]:
# Item 1
def knn_leave_one_out(datos, categorias, K):
    correctos = 0
    total = len(datos)
    ???

    return(correctos / total)

In [ ]:
# Item 2 - Aplicamos la función a los datos de pingüinos
datos = penguins[["bill_depth_mm", "bill_length_mm"]]
categorias = penguins["species"]
knn_leave_one_out(datos, categorias, 9)

**Ejercicio 4**
Utilizando las funciones de los ejercicios anteriores, calcular el valor de $K$ (impar) óptimo para predecir la especie de un pingüino.

In [ ]:
???

## Ejemplo: detección temprana de diabetes

A partir de distintos datos de pacientes queremos detectar tempranamente si ese paciente va a sufrir diabetes.

1. Leer los datos del archivo "diabetes.csv".
2. Separar la columna "Outcome" como variable respuesta y el resto como variables explicativas.
3. Escalar las variables explicativas por MinMax
4. Partir la muestra en un 80% para entrenamiento y un 20% para testeo.
5. Se quiere predecir la variable respuesta por KNN. Calcular el valor óptimo de $K$ optimizando el porcentaje de aciertos en testeo.
7. Para el valor hallado, calcular la matriz de confusión en testeo.

In [ ]:
# Utilizamos estos paquetes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [9]:
datos = pd.read_csv("../../Datos/diabetes.csv")
datos.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# 2. Separamos la variable respuesta
X = datos.drop("Outcome",axis=1)
y = datos["Outcome"]

# 3. Escalamos X
X = MinMaxScaler().set_output(transform="pandas").fit_transform(X)
X

In [ ]:
# Separamos en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
neighbor = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# Entrenamos utilizando X_train
neighbor.fit(X_train,y_train)

In [ ]:
# Predecimos utilizando X_test
y_pred = neighbor.predict(X_test)
y_pred

In [ ]:
# Calculamos la precisión con accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
# Repetimos todo para varios valores de K
for K in range(1,30,2):
    neighbor = KNeighborsClassifier(n_neighbors=K)
    neighbor.fit(X_train,y_train)
    y_pred = neighbor.predict(X_test)
    print(K, accuracy_score(y_test,y_pred))

**Matriz de confusión**
La matriz de confusión $C$ guarda en la coordenada $C_{ij}$ la cantidad de observaciones en el grupo $i$ que fueron clasificadas en el grupo $j$.

Si la variable es binaria:
- $C_{00}$ son los casos negativos clasificados correctamente.
- $C_{01}$ son los casos negativos  clasificados como positivos (falsos positivos).
- $C_{10}$ son los casos positivos  clasificados como negativos (falsos negativos).
- $C_{11}$ son los casos positivos clasificados correctamente.

In [ ]:
K = 21
neighbor = KNeighborsClassifier(n_neighbors=K)
neighbor.fit(X_train,y_train)
y_pred = neighbor.predict(X_test)

In [ ]:
C = confusion_matrix(y_test,y_pred)
display(C)

**Ejercicio:** Calcular el coeficiente $C_{01}$ a mano.

In [ ]:
sum((y_pred==1) & (y_test==0))